In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.0 MB/s 
     |████████████████████████████████| 6.6 MB 67.2 MB/s 
     |████████████████████████████████| 120 kB 67.5 MB/s 


In [2]:
import numpy as np
import pandas as pd

import sklearn.model_selection as ms
import sklearn.preprocessing as p

import tensorflow as tf
import transformers as trfs
import os

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
import tensorflow as tf
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device))

Found GPU at: /device:GPU:0


In [4]:
!nvidia-smi

Sat Sep 24 09:15:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    38W / 300W |    489MiB / 16160MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# Max length of encoded string(including special tokens such as [CLS] and [SEP]):
MAX_SEQUENCE_LENGTH = 512

# BERT model with lowercase chars only:
PRETRAINED_MODEL_NAME = 'bert-base-uncased'

tokenizer = trfs.DistilBertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

def batch_encode(X, tokenizer):
    return tokenizer.batch_encode_plus(
    X,
    max_length=MAX_SEQUENCE_LENGTH, # set the length of the sequences
    add_special_tokens=True, # add [CLS] and [SEP] tokens
    return_attention_mask=True,
    return_token_type_ids=True, # not needed for this type of ML task
    pad_to_max_length=True,
    truncation = True, # add 0 pad tokens to the sequences less than max_length
    return_tensors='tf'
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [6]:
new_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/model/sum2/summodel")

# Check its architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 bert (Custom>TFBertMainLayer)  {'pooler_output': (  109482240   ['input_ids[0][0]',              
                                None, 768),                       'attention_mask[0][0]',     

In [7]:
cdir ="/content/drive/MyDrive/Colab Notebooks/data/testdata"

In [21]:
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score, confusion_matrix
for subdir, dirs, files in os.walk(cdir):
  for file in files:
    f = os.path.join(subdir, file)
    df = pd.read_csv(f, delimiter=',', encoding= "utf-8")
    trueset=[]
    for index, row in df.iterrows():
      if (row["label2"] == 1):
        trueset.append(row["0"])
    #print(trueset)
    xtest = df["0"].values
    ytest = df["label2"]
    xtest = batch_encode(xtest, tokenizer)
    y_pred = new_model.predict(xtest.values(), batch_size=64, verbose=1)
    #print(y_pred)
    y_pred = np.argmax(y_pred, axis=1)
    #print(y_pred)
    # Print f1, precision, and recall scores
    print("Accuracy Score = " + str(accuracy_score(ytest, y_pred)))
    print("Precision Score = " + str(precision_score(ytest, y_pred , average="macro")))
    print("Recall Score = " + str(recall_score(ytest, y_pred , average="macro")))
    print("F1 Score = " + str(f1_score(ytest, y_pred , average="macro")))
    print("Confusion Matrix = " + str(confusion_matrix(ytest, y_pred)))
    #row_index=[]
    #for i in enumerate(y_pred):
      #print(i)
      #if i[1][1] >= 0.50:
        #row_index.append(i[0])
    #for j in enumerate(xtest):
      #print(j)
      #if j[0] in row_index:
        #print(j[1])

46/46 [==============================] - 34s 734ms/step
Accuracy Score = 0.9641255605381166
Precision Score = 0.4952161587526577
Recall Score = 0.48659470752089135
F1 Score = 0.4908675799086758
Confusion Matrix = [[2795   77]
 [  27    0]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


28/28 [==============================] - 20s 727ms/step
Accuracy Score = 0.9599083619702177
Precision Score = 0.5235409425997859
Recall Score = 0.5421756835369401
F1 Score = 0.5292277825711821
Confusion Matrix = [[1673   46]
 [  24    3]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


3/3 [==============================] - 2s 608ms/step
Accuracy Score = 0.8035714285714286
Precision Score = 0.48757763975155277
Recall Score = 0.49642857142857144
F1 Score = 0.47375415282392025
Confusion Matrix = [[134   6]
 [ 27   1]]


...

149/149 [==============================] - 111s 743ms/step
Accuracy Score = 0.9682422451994092
Precision Score = 0.5021972183999778
Recall Score = 0.5211640211640212
F1 Score = 0.49848810462410925
Confusion Matrix = [[9175  275]
 [  26    2]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


376/376 [==============================] - 281s 746ms/step
Accuracy Score = 0.9687994009484983
Precision Score = 0.5008002342928395
Recall Score = 0.5194259049408196
F1 Score = 0.49472808728327416
Confusion Matrix = [[23286   723]
 [   27     2]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


239/239 [==============================] - 178s 744ms/step
Accuracy Score = 0.9687048943708175
Precision Score = 0.5024513661762596
Recall Score = 0.538716689515296
F1 Score = 0.4982614451153906
Confusion Matrix = [[14762   452]
 [   25     3]]
